# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1023 15:48:34.073902    9654 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1023 15:48:38.441297    9676 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Sun Oct 23 15:48:40 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [13]:
# HOMEWORKS
# EXPERIMENT_FILE="/home/mattiacs/Documents/Developer/Edu/TUDelft/Year_4/Quarter_1/CS4215-QPECS/homeworks_2/ex6_configs/distributed_4_node.json"

# PROJECT
# EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
# EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
# CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

EXPERIMENT_FILE="../project/configs/distributed/mnist_fashionmnistcnn/epoch_25/experiment_10e-2_25.json"
CLUSTER_CONFIG="../project/configs/project_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [14]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1023 16:51:44.113287   13568 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: experiment-orchestrator: release: not found
W1023 16:51:45.058596   13582 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Sun Oct 23 16:51:47 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [7]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1023 16:47:34.274349   13165 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-23-2022 13:50:36 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-23-2022 13:50:36 root         INFO     Starting in cluster mode.
10-23-2022 13:50:36 root         INFO     Starting with experiment replication: 0 with seed: 42
10-23-2022 13:50:36 root         INFO     Starting as Orchestrator
10-23-2022 13:50:36 root         INFO     Starting Orchestrator, initializing resources....
10-23-2022 13:50:36 root         INFO     Loading in cluster configuration file
10-23-2022 13:50:36 root         INFO     Pointing configuration to in cluster configuration.
10-23-2022 13:50:36 root         INFO     Starting cluster manager
10-23-2022 13:50:36 ClusterManager INFO     Spinning up cluster manag

10-23-2022 13:53:40 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 13:53:42 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 13:53:47 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 13:53:50 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 13:53:52 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 13:53:57 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 13:54:00 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 13:54:02 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 13:54:07 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 

10-23-2022 13:57:38 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 13:57:44 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 13:57:44 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 13:57:44 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 13:57:49 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 13:57:54 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 13:57:54 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 13:57:59 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 13:58:04 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 

10-23-2022 14:01:35 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:01:38 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:01:40 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:01:45 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:01:48 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:01:49 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:01:50 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:01:55 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:01:59 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:02:00 root      

10-23-2022 14:05:31 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:05:32 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:05:36 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:05:41 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:05:43 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:05:46 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:05:51 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:05:53 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:05:53 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:05:56 root      

...

10-23-2022 14:25:11 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:25:14 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:25:14 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:25:16 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:25:21 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:25:24 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:25:26 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:25:31 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:25:34 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:25:36 root      

10-23-2022 14:29:06 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:29:08 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:29:12 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:29:17 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:29:18 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:29:18 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:29:22 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:29:27 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:29:28 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:29:32 root      

10-23-2022 14:33:02 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:33:03 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:33:08 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:33:12 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:33:13 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:33:18 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:33:22 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:33:22 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:33:23 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:33:28 root      

10-23-2022 14:36:56 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:36:59 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:37:04 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:37:06 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:37:09 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:37:14 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:37:17 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:37:19 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:37:24 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 

10-23-2022 14:40:51 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:40:55 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:41:00 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:41:01 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:41:05 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:41:10 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:41:11 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:41:15 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 0 arrivals
10-23-2022 14:41:20 root         INFO     Waiting for 8c75fbb2-7172-46a3-b9f2-64fd122ca38d to complete, 0 pending, 

In [8]:
CURRENT_POD_ID="8c75fbb2-7172-46a3-b9f2-64fd122ca38d"
# To get logs from learners (example)
kubectl logs -n test trainjob-$CURRENT_POD_ID-master-0

W1023 16:47:56.846038   13208 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-23-2022 13:52:34 root         INFO     Loading file config/configuration.fltk.json
10-23-2022 13:52:34 root         INFO     Starting in client mode
10-23-2022 13:52:34 root         INFO     Starting with host=trainjob-8c75fbb2-7172-46a3-b9f2-64fd122ca38d-master-0 and port=23456
10-23-2022 13:52:34 root         INFO     Initializing backend for training process: gloo
10-23-2022 13:52:44 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
10-23-2022 13:52:44 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-23-2022 13:52:44 root         INFO     Starting Creating client with 0
10-23-2022 13:52:44 Client-0-8

10-23-2022 14:40:31 Client-0-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [24,    50] loss: 0.111
10-23-2022 14:41:20 Client-0-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [24,   100] loss: 0.106
10-23-2022 14:41:44 Client-0-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [25,     0] loss: 0.002
10-23-2022 14:42:32 Client-0-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [25,    50] loss: 0.102
10-23-2022 14:43:21 Client-0-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [25,   100] loss: 0.104
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=115947, duration_test=8100, loss_train=0.5934267169237137, accuracy=80.92, loss=10.758625507354736, class_precision=array([0.78294574, 0.97358491, 0.59894459, 0.79109589, 0.78409091,
       0.80851064, 0.53551913, 0.95890411, 0.96902655, 0.96363636]), class_recall=array([0.74814815, 0.96268657, 0.84386617, 0.85555556, 0.54117647,
       0.99130435, 0.44343891, 0.86065574, 0.9279661 , 0.89451477]), confusion_mat=array([

       0.96956522, 0.71493213, 0.92622951, 0.97457627, 0.96624473]), confusion_mat=array([[198,   0,   5,  11,   1,   0,  53,   0,   2,   0],
       [  1, 262,   1,   3,   0,   0,   0,   0,   1,   0],
       [  2,   1, 210,   0,  20,   0,  35,   0,   1,   0],
       [  6,   2,   1, 241,   7,   0,  13,   0,   0,   0],
       [  1,   0,  18,   9, 208,   0,  19,   0,   0,   0],
       [  0,   0,   0,   1,   0, 223,   0,   5,   0,   1],
       [ 20,   0,  12,   8,  22,   0, 158,   0,   1,   0],
       [  0,   0,   0,   0,   0,  10,   0, 226,   0,   8],
       [  0,   1,   0,   2,   1,   1,   1,   0, 230,   0],
       [  0,   0,   0,   0,   0,   1,   0,   7,   0, 229]])), EpochData(epoch_id=9, num_epochs=26, duration_train=1103844, duration_test=8311, loss_train=0.24425783336162568, accuracy=87.52, loss=7.061019033193588, class_precision=array([0.86462882, 0.98867925, 0.86134454, 0.875     , 0.80933852,
       0.94871795, 0.56537102, 0.94605809, 0.97881356, 0.96624473]), class_recall=array(

       0.96956522, 0.59057971, 0.94331984, 0.97046414, 0.96610169]), class_recall=array([0.77777778, 0.97761194, 0.78066914, 0.91111111, 0.77254902,
       0.96956522, 0.73755656, 0.95491803, 0.97457627, 0.96202532]), confusion_mat=array([[210,   0,   5,  10,   1,   0,  41,   0,   3,   0],
       [  1, 262,   1,   3,   0,   0,   0,   0,   1,   0],
       [  4,   1, 210,   0,  16,   1,  35,   0,   2,   0],
       [  6,   1,   2, 246,   5,   0,  10,   0,   0,   0],
       [  0,   0,  20,  13, 197,   0,  25,   0,   0,   0],
       [  0,   0,   0,   1,   0, 223,   0,   5,   0,   1],
       [ 21,   0,  11,   8,  17,   0, 163,   0,   1,   0],
       [  0,   0,   0,   0,   0,   4,   0, 233,   0,   7],
       [  0,   1,   0,   0,   1,   2,   2,   0, 230,   0],
       [  0,   0,   0,   0,   0,   0,   0,   9,   0, 228]])), EpochData(epoch_id=17, num_epochs=26, duration_train=2073739, duration_test=8221, loss_train=0.162025545835495, accuracy=88.24, loss=7.099860817193985, class_precision=array([

       [  0,   0,   0,   0,   0,   0,   0,   9,   0, 228]])), EpochData(epoch_id=24, num_epochs=26, duration_train=2930243, duration_test=8195, loss_train=0.10553381487727165, accuracy=88.56, loss=7.893604546785355, class_precision=array([0.87351779, 0.97407407, 0.83467742, 0.89259259, 0.7992278 ,
       0.98198198, 0.64541833, 0.93227092, 0.97478992, 0.96218487]), class_recall=array([0.81851852, 0.98134328, 0.76951673, 0.89259259, 0.81176471,
       0.94782609, 0.73303167, 0.95901639, 0.98305085, 0.96624473]), confusion_mat=array([[221,   1,   7,   5,   1,   0,  32,   0,   3,   0],
       [  1, 263,   1,   1,   0,   0,   1,   0,   1,   0],
       [  4,   2, 207,   2,  21,   0,  31,   0,   2,   0],
       [  6,   3,   3, 241,  10,   0,   7,   0,   0,   0],
       [  0,   0,  19,  12, 207,   0,  17,   0,   0,   0],
       [  0,   0,   0,   1,   0, 218,   0,   9,   0,   2],
       [ 21,   0,  11,   8,  19,   0, 162,   0,   0,   0],
       [  0,   0,   0,   0,   0,   3,   0, 234,   0,   7

In [9]:
# To get logs from learners (federated learning)
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-0

W1023 16:48:11.880545   13223 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-23-2022 13:52:43 root         INFO     Loading file config/configuration.fltk.json
10-23-2022 13:52:43 root         INFO     Starting in client mode
10-23-2022 13:52:43 root         INFO     Starting with host=trainjob-8c75fbb2-7172-46a3-b9f2-64fd122ca38d-master-0 and port=23456
10-23-2022 13:52:43 root         INFO     Initializing backend for training process: gloo
10-23-2022 13:52:44 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 1
10-23-2022 13:52:44 torch.distributed.distributed_c10d INFO     Rank 1: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-23-2022 13:52:44 root         INFO

10-23-2022 14:39:43 Client-1-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [24,     0] loss: 0.003
10-23-2022 14:40:31 Client-1-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [24,    50] loss: 0.111
10-23-2022 14:41:20 Client-1-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [24,   100] loss: 0.106
10-23-2022 14:41:44 Client-1-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [25,     0] loss: 0.002
10-23-2022 14:42:32 Client-1-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [25,    50] loss: 0.102
10-23-2022 14:43:21 Client-1-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [25,   100] loss: 0.104
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=115989, duration_test=8692, loss_train=0.5934267169237137, accuracy=80.96, loss=10.23994117975235, class_precision=array([0.79912664, 0.94852941, 0.61695906, 0.79259259, 0.79166667,
       0.76368876, 0.64321608, 0.90909091, 0.95495495, 0.95454545]), class_recall=array([0.76890756, 0.95910781, 0.88284519, 0.83268482, 0.57

       0.96715328, 0.62769231, 0.93410853, 0.98253275, 0.97165992]), class_recall=array([0.74789916, 0.9739777 , 0.81171548, 0.89105058, 0.78695652,
       0.98880597, 0.80314961, 0.94509804, 0.95338983, 0.94488189]), confusion_mat=array([[178,   0,   2,   5,   0,   0,  53,   0,   0,   0],
       [  2, 262,   0,   4,   0,   0,   1,   0,   0,   0],
       [  1,   0, 194,   1,  22,   0,  21,   0,   0,   0],
       [  1,   3,   3, 229,   3,   0,  18,   0,   0,   0],
       [  0,   0,  14,  11, 181,   0,  24,   0,   0,   0],
       [  0,   0,   0,   0,   0, 265,   0,   3,   0,   0],
       [ 14,   2,  12,  10,   9,   0, 204,   0,   3,   0],
       [  0,   0,   0,   0,   0,   6,   0, 241,   1,   7],
       [  0,   1,   0,   1,   2,   3,   3,   1, 225,   0],
       [  0,   0,   0,   0,   0,   0,   1,  13,   0, 240]])), EpochData(epoch_id=9, num_epochs=26, duration_train=1103884, duration_test=8196, loss_train=0.24425783336162568, accuracy=88.56, loss=6.309998109936714, class_precision=array(

       0.96323529, 0.6503268 , 0.92193309, 0.98701299, 0.9832636 ]), class_recall=array([0.80252101, 0.9739777 , 0.80753138, 0.89883268, 0.78695652,
       0.97761194, 0.78346457, 0.97254902, 0.96610169, 0.92519685]), confusion_mat=array([[191,   0,   4,   4,   0,   0,  39,   0,   0,   0],
       [  2, 262,   0,   4,   0,   0,   1,   0,   0,   0],
       [  1,   0, 193,   2,  16,   0,  27,   0,   0,   0],
       [  3,   3,   5, 231,   2,   0,  13,   0,   0,   0],
       [  0,   0,  13,  10, 181,   0,  26,   0,   0,   0],
       [  0,   0,   0,   0,   0, 262,   0,   4,   0,   2],
       [ 19,   1,  12,   9,  10,   1, 199,   0,   3,   0],
       [  0,   0,   0,   0,   0,   5,   0, 248,   0,   2],
       [  1,   1,   1,   0,   1,   2,   1,   1, 228,   0],
       [  1,   0,   0,   0,   0,   2,   0,  16,   0, 235]])), EpochData(epoch_id=17, num_epochs=26, duration_train=2073779, duration_test=8213, loss_train=0.162025545835495, accuracy=89.24, loss=6.1958910673856735, class_precision=array(

       [  1,   0,   0,   0,   0,   0,   0,  15,   0, 238]])), EpochData(epoch_id=24, num_epochs=26, duration_train=2930280, duration_test=8198, loss_train=0.10553381487727165, accuracy=89.04, loss=6.689857646822929, class_precision=array([0.86036036, 0.98134328, 0.85201794, 0.89147287, 0.82894737,
       0.97378277, 0.65156794, 0.92509363, 0.97863248, 0.97560976]), class_recall=array([0.80252101, 0.97769517, 0.79497908, 0.89494163, 0.82173913,
       0.97014925, 0.73622047, 0.96862745, 0.97033898, 0.94488189]), confusion_mat=array([[191,   0,   4,   3,   1,   0,  38,   0,   1,   0],
       [  2, 263,   0,   3,   0,   0,   1,   0,   0,   0],
       [  1,   0, 190,   2,  21,   0,  24,   0,   1,   0],
       [  3,   3,   6, 230,   2,   0,  13,   0,   0,   0],
       [  0,   0,  10,   8, 189,   0,  23,   0,   0,   0],
       [  0,   0,   0,   0,   0, 260,   0,   6,   0,   2],
       [ 24,   1,  12,  12,  14,   1, 187,   0,   3,   0],
       [  0,   0,   0,   0,   0,   4,   0, 247,   0,   4

In [10]:
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-1

W1023 16:48:18.328715   13255 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-23-2022 13:52:44 root         INFO     Loading file config/configuration.fltk.json
10-23-2022 13:52:44 root         INFO     Starting in client mode
10-23-2022 13:52:44 root         INFO     Starting with host=trainjob-8c75fbb2-7172-46a3-b9f2-64fd122ca38d-master-0 and port=23456
10-23-2022 13:52:44 root         INFO     Initializing backend for training process: gloo
10-23-2022 13:52:44 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 2
10-23-2022 13:52:44 torch.distributed.distributed_c10d INFO     Rank 2: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-23-2022 13:52:44 root         INFO

10-23-2022 14:39:43 Client-2-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [24,     0] loss: 0.003
10-23-2022 14:40:31 Client-2-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [24,    50] loss: 0.111
10-23-2022 14:41:20 Client-2-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [24,   100] loss: 0.106
10-23-2022 14:41:44 Client-2-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [25,     0] loss: 0.002
10-23-2022 14:42:32 Client-2-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [25,    50] loss: 0.102
10-23-2022 14:43:21 Client-2-8c75fbb2-7172-46a3-b9f2-64fd122ca38d INFO     [25,   100] loss: 0.104
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=115986, duration_test=7789, loss_train=0.5934267169237137, accuracy=79.88, loss=11.159354835748672, class_precision=array([0.82142857, 0.96956522, 0.53908356, 0.76109215, 0.7740113 ,
       0.77666667, 0.61340206, 0.94759825, 0.94163424, 0.97333333]), class_recall=array([0.75409836, 0.96956522, 0.85470085, 0.88492063, 0.5

       0.95020747, 0.63522013, 0.9375    , 0.97727273, 0.96734694]), class_recall=array([0.72131148, 0.98695652, 0.79059829, 0.8968254 , 0.7816092 ,
       0.97033898, 0.75092937, 0.94488189, 0.95202952, 0.95180723]), confusion_mat=array([[176,   0,   1,  11,   3,   0,  52,   0,   1,   0],
       [  0, 227,   0,   2,   0,   0,   0,   0,   1,   0],
       [  1,   0, 185,   2,  23,   0,  23,   0,   0,   0],
       [  2,   0,   3, 226,  12,   0,   9,   0,   0,   0],
       [  0,   1,  17,  12, 204,   0,  27,   0,   0,   0],
       [  0,   0,   0,   0,   0, 229,   0,   4,   0,   3],
       [ 19,   2,  14,   8,  20,   0, 202,   0,   4,   0],
       [  0,   0,   0,   0,   0,   9,   0, 240,   0,   5],
       [  1,   0,   0,   3,   1,   2,   4,   2, 258,   0],
       [  0,   0,   0,   0,   0,   1,   1,  10,   0, 237]])), EpochData(epoch_id=9, num_epochs=26, duration_train=1103883, duration_test=8890, loss_train=0.24425783336162568, accuracy=87.28, loss=7.0131202936172485, class_precision=array

       [  0,   0,   0,   0,   0,   1,   0,  12,   0, 236]])), EpochData(epoch_id=16, num_epochs=26, duration_train=1953181, duration_test=7300, loss_train=0.16781568706035613, accuracy=88.44, loss=7.059691712260246, class_precision=array([0.85185185, 0.99122807, 0.85840708, 0.85555556, 0.84063745,
       0.958159  , 0.65448505, 0.93129771, 0.97735849, 0.97520661]), class_recall=array([0.75409836, 0.9826087 , 0.82905983, 0.91666667, 0.80842912,
       0.97033898, 0.73234201, 0.96062992, 0.95571956, 0.94779116]), confusion_mat=array([[184,   0,   2,  10,   2,   0,  45,   0,   1,   0],
       [  0, 226,   0,   3,   0,   0,   0,   0,   1,   0],
       [  1,   0, 194,   6,  14,   0,  19,   0,   0,   0],
       [  2,   0,   3, 231,   7,   0,   9,   0,   0,   0],
       [  1,   1,  13,   8, 211,   0,  27,   0,   0,   0],
       [  0,   0,   0,   0,   0, 229,   0,   4,   0,   3],
       [ 27,   1,  13,  10,  17,   0, 197,   0,   4,   0],
       [  0,   0,   0,   0,   0,   7,   0, 244,   0,   3

       [  1,   1,  13,  10, 214,   0,  22,   0,   0,   0],
       [  0,   0,   0,   0,   0, 228,   0,   5,   0,   3],
       [ 31,   0,  11,  10,  19,   0, 194,   0,   4,   0],
       [  0,   0,   0,   0,   0,   5,   0, 245,   0,   4],
       [  3,   0,   1,   1,   1,   1,   2,   2, 259,   1],
       [  0,   0,   0,   0,   0,   1,   0,  13,   0, 235]])), EpochData(epoch_id=24, num_epochs=26, duration_train=2930282, duration_test=8101, loss_train=0.10553381487727165, accuracy=88.68, loss=7.7358913868665695, class_precision=array([0.81589958, 0.99559471, 0.85903084, 0.86142322, 0.81679389,
       0.97435897, 0.69662921, 0.93181818, 0.97014925, 0.96734694]), class_recall=array([0.79918033, 0.9826087 , 0.83333333, 0.91269841, 0.81992337,
       0.96610169, 0.69144981, 0.96850394, 0.95940959, 0.95180723]), confusion_mat=array([[195,   0,   0,   8,   2,   0,  36,   0,   3,   0],
       [  0, 226,   0,   3,   0,   0,   0,   0,   1,   0],
       [  3,   0, 195,   5,  16,   0,  15,   0,   0,   

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [76]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1019 15:42:31.888203   14421 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1019 15:42:32.776003   14448 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [77]:
helm uninstall -n test experiment-orchestrator

W1019 15:42:53.020634   14486 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [78]:
helm uninstall extractor -n test

W1019 15:42:58.179859   14502 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [79]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1019 15:43:21.797054   14527 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-0203a077-858f-4427-ae7f-9c7ad5b92c71" deleted
pytorchjob.kubeflow.org "trainjob-041637b0-9be2-4a36-bc1f-0b8488d1fdd7" deleted
pytorchjob.kubeflow.org "trainjob-0a65e275-2e1f-421a-9f58-b6a809f81e41" deleted
pytorchjob.kubeflow.org "trainjob-1be7dd3d-d117-48aa-a5a2-190743e8d845" deleted
pytorchjob.kubeflow.org "trainjob-20908ee6-abae-4b99-b508-3f36dd9161ee" deleted
pytorchjob.kubeflow.org "trainjob-490cdeec-195f-45b0-94bf-fac30ce4e481" deleted
pytorchjob.kubeflow.org "trainjob-62e17e42-a74b-40b6-932d-568c69ea8c84" deleted
pytorchjob.kubeflow.org "trainjob-62ef1b19-cbb7-49d4-ae48-b74a1f16afa0" deleted
pytorchjob.kubeflow.org "trainjob-6cc6085b-4f8d-414f-b430-1e504f8a9b6f" deleted
pytorchjob.kubeflow.org "trai